In [1]:
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import time
import torch

from PIL import Image as im

In [2]:
# file_path = "/home/tigeriv/Data/Carla/Cartesian/Test/Town10_Medium/cartesian/ConvBKI/"
# file_path = "/home/tigeriv/Data/KITTI/dataset/sequences/08/ConvBKIKITTI/"
file_path = "/home/tigeriv/Data/KITTI/dataset/sequences/08/ConvBKIKITTI_Transfer/"
# eval_size = (256, 256, 64)

KITTI_COLOR = True

In [3]:
def dirichlet_to_variance(dirichlet):
    H, W, C = dirichlet.shape
    dirichlet = dirichlet.reshape(H*W, C)
    point_sums = np.sum(dirichlet, axis=-1, keepdims=False)
    bev_map = np.argmax(dirichlet, axis=-1)
    point_consts = np.amax(dirichlet, axis=-1) / point_sums
    point_vars = point_consts * (1 - point_consts) / (1 + point_sums)
    point_vars[point_sums == 0] = 0
    bev_var = point_vars.reshape(H, W)
    return bev_var

def variance_to_color(bev_var, max_var=0.15):
    H, W = bev_var.shape
    flat_var = bev_var.reshape(-1)
    # General function
    R = 255 * flat_var/max_var
    G = 255 * 0.7 * (1 - flat_var/max_var)
    B = 255 * (1 - flat_var / max_var)
    # Edge case
    too_large = flat_var >= max_var
    R[too_large] = 255
    G[too_large] = 0
    B[too_large] = 0
    unseen = flat_var <= 0
    R[unseen] = 255
    G[unseen] = 255
    B[unseen] = 255
    colors = np.stack([R, G, B], axis=1)
    colors = colors.reshape(H, W, 3).astype(np.uint8)
    return colors

In [4]:
# Form BEV
def form_bev(labels, counts):
    bev_map = np.zeros((eval_size[0], eval_size[1]))
    for x in range(eval_size[0]):
        for y in range(eval_size[1]):
            label_xy = None
            for z in range(eval_size[2]-1, -1, -1):
                count_z = counts[x, y, z]
                if count_z != 0 and labels[x, y, z] != 0:
                    intersected = True
                    label_xy = labels[x, y, z]
                    break
            if label_xy:
                remapped_xy = LABELS_REMAP[label_xy]
                bev_map[x, y] = remapped_xy
    return bev_map

def plot_bev(bev_map):
    plt.figure()
    ax = sns.heatmap(bev_map, cmap="YlGnBu", xticklabels=False, yticklabels=False)
    ax.invert_yaxis()
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    plt.show()
    
def bev_img(bev_map):
    colored_data = LABEL_COLORS[bev_map.astype(int)]
    img = im.fromarray(colored_data, 'RGB')
    return img

def var_img(var_map):
    img = im.fromarray(var_map, 'RGB')
    return img

In [5]:
save_dir = os.path.join(file_path, "VAR")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
for i in range(0, 5000):
    probs_file = os.path.join(file_path, str(i).zfill(6) + ".bin")

    bev_preds = np.fromfile(probs_file, dtype=np.float32).reshape(256, 256, 11)
    
    bev_var = dirichlet_to_variance(bev_preds)
    # plot_bev(bev_var)
    var_colored = variance_to_color(bev_var)

    img = var_img(var_colored)
    save_path = os.path.join(save_dir, str(i) + ".png")
    img.save(save_path)
    

/tmp/ipykernel_2627229/4145357697.py:6: RuntimeWarning: invalid value encountered in true_divide
  point_consts = np.amax(dirichlet, axis=-1) / point_sums


FileNotFoundError: [Errno 2] No such file or directory: '/home/tigeriv/Data/KITTI/dataset/sequences/08/ConvBKIKITTI_Transfer/004071.bin'